In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp
spark = SparkSession.builder.appName("credit_card_tracking").getOrCreate()

data = [
    (101, 1, "active", 5000.0),
    (102, 2, "active", 10000.0)
]
columns = ["card_id", "customer_id", "status", "credit_limit"]

df = spark.createDataFrame(data, columns)
credit_cards_table = "incremental_load.default.credit_cards"
df.write.format("delta") \
  .option("delta.enableChangeDataFeed", "true") \
  .mode("overwrite") \
  .saveAsTable(credit_cards_table)


In [0]:
updated_data = [
    (101, 1, "upgraded", 7000.0),   # upgraded
    (102, 2, "blocked", 10000.0)    # blocked
]
update_df = spark.createDataFrame(updated_data, columns)

from delta.tables import DeltaTable
delta_table = DeltaTable.forName(spark, credit_cards_table)\
print("Before merge")
display(delta_table)
delta_table.alias("t").merge(
    update_df.alias("s"),
    "t.card_id = s.card_id"
).whenMatchedUpdateAll().execute()
print("After merge")
display(delta_table)
